In [1]:
# Data manipulation
import pandas as pd
import numpy as np
#from pint import UnitRegistry
#u = UnitRegistry()
#u.default_format = 'C'
import math as m

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()
from IPython.display import display_latex

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import matplotlib.pyplot as plt
h = 8
w = h*1.61803398875
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 20}

plt.rc('font', **font)

#from matplotlib import rc
plt.rc('font',**{'family':'serif','serif':['Times New Roman']})
plt.rc('text', usetex=True)

# Publish
# from ipypublish.scripts import nb_setup

In [12]:
def hdf5store(filename, keys, df, **kwargs):
    store = pd.HDFStore(filename)
    store.put(keys, df, format='table', data_columns=True)
    store.get_storer('mydata').attrs.metadata = kwargs
    store.close()
    
def kb_U_eq(df,charge_config,units,blast_param,distance):
    coef_df = df[(df.charge_configuration == charge_config) &
                 (df.system_of_units == units) &
                 (df.blast_parameter == blast_param)]
    num_eq = df[(df.charge_configuration == charge_config) & # number of equations based on the distance
                (df.system_of_units == units) &              # from the charge.
                (df.blast_parameter == blast_param)].function.nunique()
    lower_range = df[(df.charge_configuration == charge_config) & # the lower distance the equation is 
                     (df.system_of_units == units) &              # valid over.
                     (df.blast_parameter == blast_param)].lower_range.unique()
    upper_range = df[(df.charge_configuration == charge_config) & # the upper range the equaiton is valid
                     (df.system_of_units == units) &              # over.
                     (df.blast_parameter == blast_param)].upper_range.unique()

    c_s = coef_df.coefficients[(coef_df.variable == 'U')] # Storing the U coefficents in the variable c_s
    u = 0
    U = 0
    Udist = []
    Uall = []
    dall = []
    func = []
    for e in range(0,num_eq): # cycle through the number of equations.
        dsub = distance[(distance>=lower_range[e]) & # filter out only the distances which are
                        (distance<=upper_range[e])]  # between the upper and lower ranges the equation is valid for.
        for T in np.log10(dsub): # cycle through only the valid (uppoer and lower ranges) distances and take the log10.
            n = 0 # resets the power in the function to zero
            for c in c_s: # cycle through the coefficients in the function.
                u = u + c*T**n # sum's the function terms together
                n += 1 # increments to the next term in the function
            Udist.append(u) # stores each total u value at each distance
            u = 0 # resets the u value to zero for the new distance dsub
        Uall.append(Udist) # stores each group of u's for upper and lower range.
        Udist = [] # resets the Udist value to zero for the next upper and lower ranges.
        dall.append(dsub) # all the distances broken into valid ranges
    flat_dall = [item for sublist in dall for item in sublist] # combine the dall distances into a one array.
    return Uall

def kb_Y_eq(df,charge_config,units,blast_param,U):
    coef_df = df[(df.charge_configuration == charge_config) &
                 (df.system_of_units == units) &
                 (df.blast_parameter == blast_param)]  
    coef = coef_df.coefficients[(coef_df.variable == 'Y')]
    
    y = 0 # set y value to zero.
    Y = [] # clear Y array
    Yall = []
    for u_ary in U: # for each u array in U
        for u in u_ary: # for each u value in u_ary
            n = 0
            for c in coef: # for each coefficient in the function
                y = y + c*u**n # totals the y's together
                n += 1 # increments to the next term in the function
            Y.append(y) # collects each total y at each distance
            u = 0 # resets the u value to zero for the new u_ary
        Yall.append(Y) # collects each group of Y's at each u and function
        Y = []
    flat_Yall = [item for sublist in Yall for item in sublist]
    return flat_Yall

In [13]:
filename = 'kb_equation_data.csv'
df = pd.read_csv(filename)
distance = np.asarray([1,10,100])
#distance = np.arange(0,101,.1)
u = kb_U_eq(df,'test','imperial','test',distance)
y = kb_Y_eq(df,'test','imperial','test',u)
u
y

[]

[]

In [8]:
fig = plt.figure(figsize=(w,h));
plt.xlabel('Distance (ft)');
plt.ylabel('Kingery Bulmash U Parameter');
plt.plot(d,y);

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f2db99b1378> (for post_execute):


FileNotFoundError: [Errno 2] No such file or directory: 'dvipng'

FileNotFoundError: [Errno 2] No such file or directory: 'dvipng'

# Airblast Parameters From TNT Spherical Air Burst and Hemispherical Surface Burst
*S. Kevin McNeill*

*August 28, 2019*

## Analysis
This notebook will plot the following blast parameters for spherical air burst and hemispherical surface burst in both imperial and metric units:
* incident pressure
* incident impulse
* reflected pressure
* reflected impulse
* shock front velocity
* arrival time
* positive phase duration


## Data
Data for these plots was built from the original paper *Airblast Parameters from TNT Spherical Air Burst and Hemispherical Surface Burst* by Charles N. Kingery and Gerald Bulmash.  It was hand typed into excel and then saved as a HDF5 file.

In [5]:
filename = 'kb_equation_data.csv'
df = pd.read_csv(filename)

In [6]:
hdf5store('kb_coefficient.h5', keys, df, **kwargs)

NameError: name 'keys' is not defined

In [ ]:
charge_config = 'hemispherical'
units = 'imperial'
blast_param = 'positive_phase_duration'
U = np.arange(0,200,0.1)

num_eq = df[(df.charge_configuration == charge_config) &
          (df.system_of_units == units) &
          (df.blast_parameter == blast_param)].function.nunique()
lower_range = df[(df.charge_configuration == charge_config) &
          (df.system_of_units == units) &
          (df.blast_parameter == blast_param)].lower_range.unique()
upper_range = df[(df.charge_configuration == charge_config) &
          (df.system_of_units == units) &
          (df.blast_parameter == blast_param)].upper_range.unique()
coef_df = df[(df.charge_configuration == charge_config) &
          (df.system_of_units == units) &
          (df.blast_parameter == blast_param)]
    
Coef = coef_df.coefficients[(coef_df.variable == 'Y')]


y = 0
Y = []
for e in range(0,num_eq):
    Unew = U[(U>=lower_range[e]) & (U<=upper_range[e])]
    for u in Unew: 
        n = 0
        for c in Coef: 
            y = y + c*u**n
            n += 1
        Y.append(u)
        u = 0

## Plots


In [ ]:
fig = plt.figure(figsize=(w,h));
plt.xlabel('Distance (ft)');
plt.ylabel('Kingery Bulmash U Parameter');
plt.plot(distance,U, 'ro', linestyle = '-');

In [ ]:
coef_df = df[(df.charge_configuration == charge_config) &
             (df.system_of_units == units) &
             (df.blast_parameter == blast_param)]

Coef = coef_df.coefficients[(coef_df.variable == 'Y')]
y = 0
Y = []
for u in U: 
    n = 0
    for c in Coef: 
        y = y + c*u**n
        print(n,c,u,c*u**n)
        n += 1
    Y.append(y)
    u = 0
print(U,Y)

## References